# Project 1 ---  IMAGE CLASSIFICATION

---



## We have experimented with various Deep Learning models, like *VGG, Densenet, ResNet, Inception, and their ensembles*

### We have found pretrained inception-V3 model provided in the standard torch library to have highest categorization accuracy., thereby achieving 80.73% accuracy on test data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os, glob, sys
import copy
from shutil import move
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.1+cu101
Torchvision Version:  0.6.1+cu101


In [ ]:
##unzipping the zip file into the current working folder

!unzip "/content/drive/My Drive/image-detect.zip"

Streaming output truncated to the last 5000 lines.
  inflating: val/images/val_5501.JPEG  
  inflating: val/images/val_5502.JPEG  
  inflating: val/images/val_5503.JPEG  
  inflating: val/images/val_5504.JPEG  
  inflating: val/images/val_5505.JPEG  
  inflating: val/images/val_5506.JPEG  
  inflating: val/images/val_5507.JPEG  
  inflating: val/images/val_5508.JPEG  
  inflating: val/images/val_5509.JPEG  
  inflating: val/images/val_551.JPEG  
  inflating: val/images/val_5510.JPEG  
  inflating: val/images/val_5511.JPEG  
  inflating: val/images/val_5512.JPEG  
  inflating: val/images/val_5513.JPEG  
  inflating: val/images/val_5514.JPEG  
  inflating: val/images/val_5515.JPEG  
  inflating: val/images/val_5516.JPEG  
  inflating: val/images/val_5517.JPEG  
  inflating: val/images/val_5518.JPEG  
  inflating: val/images/val_5519.JPEG  
  inflating: val/images/val_552.JPEG  
  inflating: val/images/val_5520.JPEG  
  inflating: val/images/val_5521.JPEG  
  inflating: val/images/val_552

In [ ]:
## modifying the directory structure of validation folder
'''
val/
    class_X/
        //imageXX1.jpeg
        //imageXX2.jpeg
        .
        .
    class_Y/
        //imageYY1.jpeg
        .
        .
    .
    .
'''

val_dic = {}
f = open("/content/val/val_annotations.txt", "r")
for line in f.readlines():
  l = line.split('\t')
  val_dic[l[0]] = l[1]

path = glob.glob("/content/val/images/*")
for p in path:
  file = p.split("/")[-1]
  folder = val_dic[file]
  if not os.path.exists("/content/val/" + str(folder)):
        os.mkdir("/content/val/" + str(folder)) 

for p in path:
  file = p.split("/")[-1]
  folder = val_dic[file]
  dest = "/content/val/" + str(folder) + '/' + str(file)
  move(p, dest)
f.close()
os.rmdir('/content/val/images')
os.remove("/content/val/val_annotations.txt")

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "/content/"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception"

# Number of classes in the dataset
num_classes = 200

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for
num_epochs = 5

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [ ]:
## training function

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i,(inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                print("\rIteration: {}/{}, Loss: {}.".format(i+1, len(dataloaders[phase]), loss.item() * inputs.size(0)),  end="")

                print( (i+1)*100. / len(dataloaders[phase]), "% Complete" )
                sys.stdout.flush()
                

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('{} time elapsed'.format(time.time()-epoch_start))
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
## setting for Fine-Tuning vs Feature-Extraction

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
## You can choose between the model to experiment with
## Adding final dense layer to each model

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        model_ft.avgpool =nn.AdaptiveAvgPool2d(1)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size



In [ ]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
#model_ft_2, input_size_2 = initialize_model('resnet', num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
#print(model_ft)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth


In [ ]:
## Data Augmentation and Data Loader

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=32) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Initializing Datasets and Dataloaders...


In [ ]:
'''
class_list[label_encoding] = class
'''

class_list = image_datasets['train'].classes

In [ ]:
## Optimizer (SGD)

# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.6)

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=9, gamma=0.1)


In [ ]:
## Training the Model

model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, exp_lr_scheduler,  num_epochs=3, is_inception=(model_name=="inception"))

In [ ]:
## Evaluating on Test Images
'''
storing the result in
  dict(result) = {
    img : label
  }

'''

test_dir = "/content/test/images/*"

from PIL import Image
preprocess = transforms.Compose([transforms.Resize((299,299), 3), transforms.ToTensor()])
#preprocess_2 = transforms.Compose([transforms.Resize((224,224), 3), transforms.ToTensor()])
normalise = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
path = glob.glob(test_dir)
result = {}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft.to(device)
#model_ft_2.to(device)


for p in path:
 
  img = p.split('/')[-1]
  image = Image.open(p)
  image_tensor = preprocess(image)
  #image_tensor_2 = preprocess_2(image)
  
  if image_tensor.shape == (1,299,299):
    
    img_tensr = torch.zeros([3,299,299])
    img_tensr[0,:,:] = image_tensor
    img_tensr[1,:,:] = image_tensor
    img_tensr[2,:,:] = image_tensor
    image_tensor = img_tensr
  
  


  image_batch = normalise(image_tensor).unsqueeze(0).to(device)
  model_ft.eval()
  output = model_ft(image_batch)

  
  _, predicted = torch.max(output, 1)
  result[img] = [class_list[predicted]]


In [ ]:
## converting dict(result) to csv file

import csv
a_file = open("/content/result_ens.csv", "w")

writer = csv.writer(a_file)
writer.writerow(['file_name', 'category'])
for key, value in result.items():
    writer.writerow([key, value])

a_file.close()


In [ ]:
import pandas as pd
df = pd.read_csv("/content/result_ens.csv")

In [ ]:
df["category"].value_counts()


['n02909870']    89
['n02769748']    74
['n04275548']    73
['n04560804']    70
['n04507155']    69
                 ..
['n02963159']    38
['n03670208']    38
['n03089624']    37
['n03706229']    35
['n07711569']    25
Name: category, Length: 200, dtype: int64